In [6]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 07 11:41:15 2021

@author: ovsicori
"""

#! /usr/bin/env python
# -*- coding: utf-8 -*-
#################################################################################################
#### MOVING AVERAGE
#################################################################################################

import numpy as np
from pandas import Series, DataFrame
import datetime
import warnings
from io import StringIO
warnings.filterwarnings("ignore",category=UserWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)
import hvplot.pandas  # noqa
import urllib.request
from scipy import signal
## Bokeh
### MODULE GLOBAL VARIABLES AND FUNCTIONS
#################################################################################################

ModuleNotFoundError: No module named 'scipy'

In [2]:
url = 'https://github.com/Mathilda000/OVSICORI_Geodesy'
filename = 'BIJA.ovs.final_igb14.pos'
posfinal = urllib.request.urlopen('https://raw.githubusercontent.com/Mathilda000/OVSICORI_Geodesy/main/BIJA.ovs.final_igb14.pos')

In [3]:
# Pos file header size
posheadsize = 37

# Number of days in the moving window
lag = 28

if filename.endswith(".pos"):
    fileposkf = np.loadtxt(filename, dtype = [('YYYYMMDD', 'i4'), ('HHMMSS', 'i4'), ('JJJJJ.JJJJ', 'float'), ('X', 'float'), ('Y', 'float'), ('Z', 'float'), ('Sx', 'float'), ('Sy', 'float'), ('Sz', 'float'), ('Rxy', 'float'), ('Rxz', 'float'), ('Ryz', 'float'), ('NLat', 'float'), ('Elong', 'float'), ('Height', 'float'), ('dN', 'float'), ('dE', 'float'), ('dU', 'float'), ('Sn', 'float'), ('Se', 'float'), ('Su', 'float'), ('Rne', 'float'), ('Rnu', 'float'), ('Reu', 'float'), ('Soln', 'S5')],skiprows=posheadsize)
    llaves = ['YYYYMMDD','HHMMSS','JJJJJ.JJJJ','X','Y','Z','Sx','Sy','Sz','Rxy','Rxz','Ryz','NLat','Elong','Height','dN','dE','dU','Sn','Se','Su','Rne','Rnu','Reu']
    datos = np.transpose(np.array([fileposkf[llave] for llave in llaves]))

    posfirstdate = datetime.datetime.strptime(str(fileposkf['YYYYMMDD'][0]), '%Y%m%d').date()
    poslastdate = datetime.datetime.strptime(str(fileposkf['YYYYMMDD'][-1]), '%Y%m%d').date()
    n_timesteps = (poslastdate-posfirstdate).days + 1

    observations = np.ma.masked_all((n_timesteps, 25))

    fechas_diarias = np.array([posfirstdate+datetime.timedelta(days=i) for i in range(n_timesteps)])
    fechas_datos = np.array([datetime.datetime.strptime(str(fecha), '%Y%m%d').date() for fecha in fileposkf['YYYYMMDD']])
    indices = np.in1d(fechas_diarias,fechas_datos)
    observations[indices,0:-1] = datos[:,:]

    data_moving_avg = np.ma.masked_all((n_timesteps, 25))
    data_moving_avg[:,0] = np.array([int(date.strftime('%Y%m%d')) for date in fechas_diarias]) # date
    data_moving_avg[:,1] = np.array([datos[0,1] for i in range(n_timesteps)]) # hour
    data_moving_avg[:,2] = np.array([datos[0,2]+i for i in range(n_timesteps)]) # julian day
    indices = np.logical_not(observations[:,12] == np.ma.masked)
    data_moving_avg[indices,12] = observations[indices,12] # NLat
    data_moving_avg[indices,13] = observations[indices,13] # Elong
    data_moving_avg[indices,14] = observations[indices,14] # Height

#        observations[:,15] = Series(observations[:,15]).interpolate().values # interpolating nans values of dN, dE, dU
#        observations[:,16] = Series(observations[:,16]).interpolate().values
#        observations[:,17] = Series(observations[:,17]).interpolate().values
#        
#        if (lag%2) == 0:
#            data_moving_avg[:,15] = shift4(np.convolve((np.convolve(observations[:,15], np.ones(lag), 'same') / lag), np.ones(2), 'same') / 2, -1, np.nan)
#            data_moving_avg[:,16] = shift4(np.convolve((np.convolve(observations[:,16], np.ones(lag), 'same') / lag), np.ones(2), 'same') / 2, -1, np.nan)
#            data_moving_avg[:,17] = shift4(np.convolve((np.convolve(observations[:,17], np.ones(lag), 'same') / lag), np.ones(2), 'same') / 2, -1, np.nan)
#        else:
#            data_moving_avg[:,15] = np.convolve(observations[:,15], np.ones(lag), 'same') / lag
#            data_moving_avg[:,16] = np.convolve(observations[:,16], np.ones(lag), 'same') / lag
#            data_moving_avg[:,17] = np.convolve(observations[:,17], np.ones(lag), 'same') / lag

    data_moving_avg_1s = data_moving_avg.copy()
    data_moving_avg_w = data_moving_avg.copy()
    data_moving_avg_e = data_moving_avg.copy()
    
    weights = np.array([1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1, 1, 1, 1, 1, 1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])/20 # add up to 1
    sum_weights = np.sum(weights)

    if lag%2 == 0:            
        data_moving_avg[:,15] = Series(observations[:,15]).rolling(window = lag, min_periods = 0, center = True).mean().rolling(2, min_periods = 0, center = True).mean().shift(-1).values # perfectly centered moving average of dN, dE, dU, because pandas' rolling isn't really centered and shifts forward
        data_moving_avg[:,16] = Series(observations[:,16]).rolling(window = lag, min_periods = 0, center = True).mean().rolling(2, min_periods = 0, center = True).mean().shift(-1).values
        data_moving_avg[:,17] = Series(observations[:,17]).rolling(window = lag, min_periods = 0, center = True).mean().rolling(2, min_periods = 0, center = True).mean().shift(-1).values
        
    else:
        data_moving_avg[:,15] = Series(observations[:,15]).rolling(window = lag, min_periods = 0, center = True).mean().values
        data_moving_avg[:,16] = Series(observations[:,16]).rolling(window = lag, min_periods = 0, center = True).mean().values
        data_moving_avg[:,17] = Series(observations[:,17]).rolling(window = lag, min_periods = 0, center = True).mean().values
    
    data_moving_avg_1s[:,15] = Series(observations[:,15]).rolling(window = lag, min_periods = 0).mean().values
    data_moving_avg_1s[:,16] = Series(observations[:,16]).rolling(window = lag, min_periods = 0).mean().values
    data_moving_avg_1s[:,17] = Series(observations[:,17]).rolling(window = lag, min_periods = 0).mean().values

    std_pos = np.array([data_moving_avg_1s[:,15] + Series(observations[:,15]).rolling(window = lag, min_periods = 0).std().values, data_moving_avg_1s[:,16] + Series(observations[:,16]).rolling(window = lag, min_periods = 0).std().values, data_moving_avg_1s[:,17] + Series(observations[:,17]).rolling(window = lag, min_periods = 0).std().values]).transpose()
    std_neg = np.array([data_moving_avg_1s[:,15] - Series(observations[:,15]).rolling(window = lag, min_periods = 0).std().values, data_moving_avg_1s[:,16] - Series(observations[:,16]).rolling(window = lag, min_periods = 0).std().values, data_moving_avg_1s[:,17] - Series(observations[:,17]).rolling(window = lag, min_periods = 0).std().values]).transpose()
    
    data_moving_avg_w[:,15] = Series(observations[:,15]).rolling(window = lag, min_periods = 0, win_type = 'triang').mean().values
    data_moving_avg_w[:,16] = Series(observations[:,16]).rolling(window = lag, min_periods = 0, win_type = 'triang').mean().values
    data_moving_avg_w[:,17] = Series(observations[:,17]).rolling(window = lag, min_periods = 0, win_type = 'triang').mean().values

    data_moving_avg_e[:,15] = Series(observations[:,15]).ewm(span=10, min_periods = int(lag/2), adjust = False).mean().values
    data_moving_avg_e[:,16] = Series(observations[:,16]).ewm(span=10, min_periods = int(lag/2), adjust = False).mean().values
    data_moving_avg_e[:,17] = Series(observations[:,17]).ewm(span=10, min_periods = int(lag/2), adjust = False).mean().values
    
    # axe1 = DataFrame({'dN':observations[:,15]}).reset_index().hvplot(x = 'index', y = 'dN', figsize = (15,5), title = 'dN', kind = 'scatter', s=2)
    # DataFrame({'dN_ma':data_moving_avg[:,15]}).hvplot(figsize = (15,5), kind = 'line', color = 'orange', ax = axe1, grid = True)
    display(DataFrame({'dN_ma':data_moving_avg[:,15]}).hvplot(figsize = (15,5), kind = 'line', color = 'orange',  grid = True, label = 'Centered') 
            * DataFrame({'dN':observations[:,15]}).reset_index().hvplot(x = 'index', y = 'dN', figsize = (15,5), title = 'dN', kind = 'scatter', label = 'Data') 
            * DataFrame({'dN_ma_one-sided':data_moving_avg_1s[:,15]}).hvplot(figsize = (15,5), kind = 'line', color = 'green',  grid = True, label = 'One-sided') 
            * DataFrame({'dN_ma_w':data_moving_avg_w[:,15]}).hvplot(figsize = (15,5), kind = 'line', color = 'purple',  grid = True, label = 'Weighted')
            * DataFrame({'dN_ma_e':data_moving_avg_e[:,15]}).hvplot(figsize = (15,5), kind = 'line', color = 'red',  grid = True, label = 'Exponential'))
    
    # axe2 = DataFrame({'dE':observations[:,16]}).reset_index().hvplot(x = 'index', y = 'dE', figsize = (15,5), title = 'dE', kind = 'scatter', s=2)
    # DataFrame({'dE_ma':data_moving_avg[:,16]}).hvplot(figsize = (15,5), kind = 'line', color = 'orange',  ax = axe2, grid = True)
    display(DataFrame({'dE_ma':data_moving_avg[:,16]}).hvplot(figsize = (15,5), kind = 'line', color = 'orange',  grid = True, label = 'Centered') 
            * DataFrame({'dE':observations[:,16]}).reset_index().hvplot(x = 'index', y = 'dE', figsize = (15,5), title = 'dE', kind = 'scatter', label = 'Data') 
            * DataFrame({'dE_ma_one-sided':data_moving_avg_1s[:,16]}).hvplot(figsize = (15,5), kind = 'line', color = 'green',  grid = True, label = 'One-sided')
            * DataFrame({'dE_ma_w':data_moving_avg_w[:,16]}).hvplot(figsize = (15,5), kind = 'line', color = 'purple',  grid = True, label = 'Weighted')
            * DataFrame({'dE_ma_e':data_moving_avg_e[:,16]}).hvplot(figsize = (15,5), kind = 'line', color = 'red',  grid = True, label = 'Exponential'))
        
    # axe3 = DataFrame({'dU':observations[:,17]}).reset_index().hvplot(x = 'index', y = 'dU', figsize = (15,5), title = 'dU', kind = 'scatter', s=2)
    # DataFrame({'dU_ma':data_moving_avg[:,17]}).hvplot(figsize = (15,5), kind = 'line', color = 'orange',  ax = axe3, grid = True)   
    display(DataFrame({'dU_ma':data_moving_avg[:,17]}).hvplot(figsize = (15,5), kind = 'line', color = 'orange',  grid = True, label = 'Centered') 
            * DataFrame({'dU':observations[:,17]}).reset_index().hvplot(x = 'index', y = 'dU', figsize = (15,5), title = 'dU', kind = 'scatter', label = 'Data') 
            * DataFrame({'dU_ma_one-sided':data_moving_avg_1s[:,17]}).hvplot(figsize = (15,5), kind = 'line', color = 'green',  grid = True, label = 'One-sided')
            * DataFrame({'dU_ma_w':data_moving_avg_w[:,17]}).hvplot(figsize = (15,5), kind = 'line', color = 'purple',  grid = True, label = 'Weighted')
            * DataFrame({'dU_ma_e':data_moving_avg_e[:,17]}).hvplot(figsize = (15,5), kind = 'line', color = 'red',  grid = True, label = 'Exponential'))
    
    display(DataFrame({'dN_ma_one-sided':data_moving_avg_1s[:,15]}).hvplot(figsize = (15,5), kind = 'line', color = 'green',  grid = True, label = 'One-sided')
            * DataFrame({'dN':observations[:,15]}).reset_index().hvplot(x = 'index', y = 'dN', figsize = (15,5), title = 'dN with standard deviation', kind = 'scatter', label = 'Data') 
            * DataFrame({'dN_std_pos':std_pos[:,0]}).hvplot(figsize = (15,5), kind = 'line', color = 'brown',  grid = True, label = 'Standard deviation up')
            * DataFrame({'dN_std_neg':std_neg[:,0]}).hvplot(figsize = (15,5), kind = 'line', color = 'orange',  grid = True, label = 'Standard deviation down'))

ImportError: Missing optional dependency 'scipy.signal'. Scipy is required to generate window weight. Use pip or conda to install scipy.signal.

In [7]:
import os
print(os.getcwd())

/home/jovyan


In [4]:
%watermark --iversions

UsageError: Line magic function `%watermark` not found.
